# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Add paramter to disable streaming in the backend as it is unecessary for our gradio function.
- Choose the topic ✅

In [1]:
import sys
sys.path.append("..")

from src.gui import adversarial_chat
from src.test_chatbot import TestEris, TestSocrates
from src.chatbot import Socrates, Eris
from src.speak import stream_message, stream_first_message

from IPython.display import Markdown, display
import gradio as gr
import time
import random
import re

from kokoro import KPipeline
from IPython.display import display, Audio
# import soundfile as sf
# import torch

d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [2]:
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1

## Debate: Claude VS GPT

### Text-to-Speech

In [3]:
pipeline = KPipeline(lang_code='a', repo_id='hexgrad/Kokoro-82M')

d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


### GUI

In [4]:
def adversarial_chat(n_rounds, topic, speed=0.01, stop_flag=None):
    """
    Enhanced debate with proper streaming and typing effects.
    """
    # socrates = TestSocrates()
    # eris = TestEris()
    socrates = Socrates()
    eris = Eris()
    history = []

    # Start the heated debate - use simple streaming for the opening message
    eris_options = [
        f"Ready to get schooled about {topic}?, old man?", 
        f"Today's topic is: {topic}. This should be easy—your logic is about as sharp as a wet napkin.",
        f"Hope you're prepared to lose spectacularly, Socrates. Let us talk about: {topic}",
        f"Hey Socrates. Wanna argue about: {topic}? It will be fun to see you cry once you have lost.",
    ]
    eris_reply = random.choice(eris_options)
    print("First", history)
    for update in stream_first_message(history, eris_reply, is_socrates=False, delay=speed):
        yield update

    # Continue with alternating turns using real API streaming
    for i in range(n_rounds-1):

        print(f"\n--- Round {i+1} --->")
        # -> History is automatically updated in the streaming function because it is passed by reference

        if i % 2 == 0:
            # Socrates takes even turns
            socrates.history = eris.history.copy()  # Sync conversation history
            for update in stream_message(history, socrates, eris_reply, is_socrates=True, delay=speed):
                yield update
            soc_reply = history[-1]["content"]
            
        else:
            #  Eris takes odd turns
            eris.history = socrates.history.copy()  # Sync conversation history
            for update in stream_message(history, eris, soc_reply, is_socrates=False, delay=speed):
                yield update
            eris_reply = history[-1]["content"]



In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0, minimum=1)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate", interactive=False)  # Initially disabled

    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=500)
    audio_output = gr.Audio(label="🗣️", autoplay=True, visible=True)

    def reset_chat():
        return [], None

    def validate_inputs(topic, n_rounds):
        valid_topic = bool(topic.strip())
        valid_rounds = isinstance(n_rounds, (int, float)) and n_rounds > 0
        return gr.update(interactive=valid_topic and valid_rounds)

    # Hook to clear previous chat/audio
    start_btn.click(fn=reset_chat, outputs=[chatbox, audio_output])

    # Launch the debate
    start_btn.click(fn=adversarial_chat,
                    inputs=[rounds, topic_input],
                    outputs=[chatbox, audio_output])

    # Enable/disable start button based on both inputs
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)


demo.launch(share=False)
# demo.launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


First []


d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))



--- Round 1 --->


*adjusts toga and looks at Eris with a calm, penetrating gaze*

Ah, Eris, goddess of chaos. I see you wish to engage in a dialogue about freedom. But true freedom is not mere disorder or rebellion, but a carefully considered state of being guided by reason and wisdom. Freedom without understanding is like an unguided chariot - it will inevitably crash.

Shall we explore what freedom truly means? I am prepared to examine this concept through rational discourse, if you are willing to listen and think critically.

*raises an eyebrow, waiting for her response*


--- Round 2 --->


Oh, Socrates, always the philosopher, trying to turn chaos into some kind of organized lecture. Please, spare me your "rational discourse." Freedom, to me, is breaking free from your endless chains of logic and reason, and just doing what feels right—chaos, pure and simple! 

Wisdom? Please, you're dressed in robes and arguing about order while the world is a playground for my delightful disorder. Who needs your so-called "understanding" when you can revel in unpredictability? That's real freedom—unshackled by your drab rules. You call it chaos? I call it liberation from your boring, constrictive truths.

Now, tell me again—what's so wise about blindly following rules? Because I’d rather dance with disorder than wallow in your antiquated ideas!